# Logistic Regression & Calibration — Student Lab

We focus on *probabilities*, not just accuracy.

In [1]:
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic imbalanced data
We simulate logits and labels with imbalance and miscalibration.

In [2]:
def make_probs(n=5000, base_rate=0.05, logit_scale=1.0, miscalibration=1.0):
    # Generate true probabilities via latent logits
    z = logit_scale * rng.standard_normal(n)
    # shift to get desired base rate approximately
    z = z + np.log(base_rate/(1-base_rate))
    p_true = 1/(1+np.exp(-z))
    y = (rng.random(n) < p_true).astype(int)
    # observed model probs are miscalibrated by scaling logits
    z_model = miscalibration * z
    p_model = 1/(1+np.exp(-z_model))
    return y, p_model, p_true

y, p_model, p_true = make_probs(miscalibration=2.0)
print('base_rate', y.mean())
check('in_0_1', np.all((p_model>=0) & (p_model<=1)))

base_rate 0.0712
OK: in_0_1


## Section 1 — Metrics

### Task 1.1: Confusion matrix metrics at a threshold
Implement precision, recall, F1 at threshold t.

# HINT:
- y_hat = (p>=t)
- TP/FP/FN

**Checkpoint:** Why is accuracy misleading under imbalance?

**Answer:** Accuracy counts all correct predictions equally. Under class imbalance (for instance, 99% negatives, 1% positives), a trivial model that predicts everything as negative achieves 99% accuracy, yet it completely fails at the task of finding positives (TP = 0, recall = 0). Accuracy is dominated by the majority class (TN), so it hides poor performance on the minority class that usually matters.

In [3]:
def metrics_at_threshold(y, p, t):
    # TODO
    y = y.astype(int)
    y_hat = (p >= t).astype(int)
    tp = int(np.sum((y_hat == 1) & (y == 1)))
    fp = int(np.sum((y_hat == 1) & (y == 0)))
    fn = int(np.sum((y_hat == 0) & (y == 1)))
    # tn = int(np.sum((y_hat == 0) && (y == 0)))

    precision = tp / (tp + fp + 1e-12)
    recall = tp / (tp + fn + 1e-12)
    F1_score = 2 * precision * recall / (precision + recall + 1e-12)
    return {'tp':tp, 'fp':fp, 'fn':fn, 'recall':recall, 'precision':precision, 'F1 score':F1_score }

m = metrics_at_threshold(y, p_model, t=0.5)
print(m)

{'tp': 2, 'fp': 2, 'fn': 354, 'recall': 0.005617977528089872, 'precision': 0.499999999999875, 'F1 score': 0.011111111111089075}


### Task 1.2: PR curve area (approx)
Compute a simple PR-AUC approximation by sorting thresholds.

# HINT:
- sort by p desc
- compute precision/recall at each cut

**Interview Angle:** when is PR-AUC preferable to ROC-AUC?

**Answer:** PR-AUC is preferable to ROC-AUC when the positive class is rare and we care about performance on positives (finding true cases and avoiding false alarms).
- Severe class imbalance
- False positives are costly
- We care about “positive usefulness”
- PR-AUC reflects real trade-offs between precision and recall WHERE AS ROC-AUC can look good even when no usable threshold exists.

In [11]:
def pr_curve(y, p):
    # TODO: return arrays (recall, precision)
    order = np.argsort(-p)
    y_sorted = y[order]
    tp = np.cumsum(y_sorted == 1)
    fp = np.cumsum(y_sorted == 0)
    # print(tp, fp)
    precision = tp / (tp + fp + 1e-12)
    recall = tp / (tp[-1] + 1e-12)
    return recall, precision

def auc_trapz(x, y):
    # TODO
    return float(np.trapezoid(y, x))

rec, prec = pr_curve(y, p_model)
pr_auc = auc_trapz(rec, prec)
print('pr_auc', pr_auc)
check('finite', np.isfinite(pr_auc))

pr_auc 0.20170158178004188
OK: finite


## Section 2 — Calibration

### Task 2.1: Reliability curve + ECE

Bin probabilities and compute:
- avg predicted prob per bin
- empirical accuracy per bin
- ECE = sum (bin_weight * |acc - conf|)

# HINT:
- np.digitize

**FAANG gotcha:** model can have good ranking but bad calibration.

In [16]:
def reliability_bins(y, p, n_bins=10):
    # TODO: return (bin_acc, bin_conf, bin_frac)
    y = y.astype(int)
    edges = np.linspace(0, 1, n_bins + 1)
    b = np.digitize(p, edges[1:-1], right=False)

    bin_accuracy = np.zeros(n_bins)
    bin_configuration = np.zeros(n_bins)
    bin_fraction = np.zeros(n_bins)

    for i in range(n_bins):
      mask = (b == i)
      if mask.any():
        bin_accuracy[i] = y[mask].mean()
        bin_configuration[i] = p[mask].mean()
        bin_fraction[i] = mask.mean()

    return bin_accuracy, bin_configuration, bin_fraction


def ece(bin_acc, bin_conf, bin_frac):
    # TODO
    return float(np.sum(bin_frac * np.abs(bin_acc - bin_conf)))

bin_acc, bin_conf, bin_frac = reliability_bins(y, p_model, n_bins=10)
ECE = ece(bin_acc, bin_conf, bin_frac)
print('ECE', ECE)
check('ece_finite', np.isfinite(ECE))

ECE 0.056267450617459955
OK: ece_finite


### Task 2.2: Temperature scaling

We assume p_model came from logits z_model. Approximate logits via logit(p).
Then find temperature T that minimizes NLL on validation split: sigmoid(z/T).

# HINT:
- logit(p)=log(p/(1-p))
- grid search T over [0.5..5]

**Checkpoint:** Why does scaling logits preserve ranking?

In [21]:
def logit(p, eps=1e-12):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))

def nll(y, p, eps=1e-12):
    p = np.clip(p, eps, 1-eps)
    return float(-np.mean(y*np.log(p) + (1-y)*np.log(1-p)))

# TODO: split into val and fit T
idx = rng.permutation(len(y))
val = idx[: len(y)//2]
test = idx[len(y)//2:]

z = logit(p_model)

Ts = np.linspace(0.5, 5.0, 50)
best_T = None
best_loss = float('inf')
for T in Ts:
  pT = 1 / (1 + np.exp(-(z[val] / T)))
  L = nll(y[val], pT)
  if L < best_loss:
    best_loss = L
    best_T = T
# p_cal = 1 / (1 + np.exp(-z[test] / best_T))

print('best_T', best_T)
# apply temperature on test
p_cal = 1 / (1 + np.exp(-(z[test] / best_T)))
ECE_before = ece(*reliability_bins(y[test], p_model[test], 10))
ECE_after = ece(*reliability_bins(y[test], p_cal, 10))
print('ECE_before', ECE_before, 'ECE_after', ECE_after)

best_T 2.0612244897959187
ECE_before 0.051093775041383536 ECE_after 0.014975308183525216


## Section 3 — Thresholding with costs

### Task 3.1: Pick threshold minimizing cost
Cost = c_fp*FP + c_fn*FN

# TODO: sweep thresholds and pick best.

**Interview Angle:** map model probabilities to business decisions.

In [22]:
def best_threshold_cost(y, p, c_fp=1.0, c_fn=10.0):
    # TODO
    ts = np.linspace(0, 1, 501)
    best_T = 0.5
    best_cost = float('inf')

    for t in ts:
      y_hat = (p >= t).astype(int)
      fp = np.sum((y_hat == 1) & (y == 0))
      fn = np.sum((y_hat == 0) & (y == 1))
      cost = c_fp * fp + c_fn * fn

      if cost < best_cost:
        best_cost = cost
        best_T = float(t)

    return best_T, float(best_cost)

t_star, cost_star = best_threshold_cost(y, p_model, c_fp=1.0, c_fn=10.0)
print('t*', t_star, 'cost', cost_star)

t* 0.006 cost 2626.0


---
## Submission Checklist
- All TODOs completed
- ECE computed
- Temperature scaling applied
- Threshold recommendation written
